In [1]:
# 📦 Imports and setup
import json
import yfinance as yf
from datetime import datetime, timedelta
from pathlib import Path

# Load last logged trade
log_path = Path("data/trade_log.json")
with open(log_path, "r") as f:
    trades = json.load(f)

last_trade = trades[-1]
print("🧾 Last Trade:\n", last_trade)


🧾 Last Trade:
 {'timestamp': '2025-04-21T22:59:16.189413', 'ticker': 'SPY', 'strategy_name': 'Covered Call', 'description': "The Covered Call strategy ranks first due to its ability to generate income and limit downside risk, which aligns with the trader's strengths. It also adapts well to a neutral or slightly bullish market, providing flexibility in various market conditions.", 'entry_price': 3.25, 'dte': 14, 'target_pct': 0.25, 'stop_pct': 0.15, 'status': 'OPEN'}


In [2]:
# 📊 Pull recent price data
ticker = last_trade["ticker"]
entry_price = last_trade["entry_price"]
dte = last_trade["dte"]
target_pct = last_trade["target_pct"]
stop_pct = last_trade["stop_pct"]

target_price = entry_price * (1 + target_pct)
stop_price = entry_price * (1 - stop_pct)

# 🗓️ Pull recent daily candles from yfinance (14 days)
ohlc = yf.download(ticker, period="14d", interval="1d")
ohlc = ohlc.reset_index()

print("📈 Simulated Price Feed:\n")
display(ohlc[["Date", "High", "Low", "Close"]].head())


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

📈 Simulated Price Feed:



Price,Date,High,Low,Close
Ticker,,SPY,SPY,SPY
0,2025-04-02,567.419983,554.809998,564.520020
1,2025-04-03,547.969971,536.700012,536.700012
2,2025-04-04,525.869995,505.059998,505.279999
3,2025-04-07,523.169983,481.799988,504.380005
4,2025-04-08,524.979980,489.160004,496.480011


In [3]:
# 🔁 Simulate trade outcome over daily candles
outcome = "Open"
exit_price = None
exit_date = None

for i, row in ohlc.iterrows():
    high = row[("High", ticker)]
    low = row[("Low", ticker)]
    date = row[("Date", "")]

    if high >= target_price:
        outcome = "Target Hit"
        exit_price = target_price
        exit_date = date.strftime("%Y-%m-%d")
        break
    elif low <= stop_price:
        outcome = "Stop Hit"
        exit_price = stop_price
        exit_date = date.strftime("%Y-%m-%d")
        break
    else:
        dte -= 1
        if dte <= 0:
            outcome = "Expired"
            exit_price = row[("Close", ticker)]
            exit_date = date.strftime("%Y-%m-%d")
            break

# 📦 Attach result to trade
last_trade["status"] = outcome
last_trade["exit_price"] = exit_price
last_trade["exit_date"] = exit_date
last_trade["dte"] = dte

print(f"📆 Trade Completed: {outcome} on {exit_date or 'N/A'} at ${exit_price:.2f}")


📆 Trade Completed: Target Hit on 2025-04-02 at $4.06


In [4]:
# 💾 Save updated trade log
log_path = Path("data/trade_log.json")

# Load existing log
with open(log_path, "r") as f:
    trades = json.load(f)

# Replace last trade with updated one
trades[-1] = last_trade

# Save it back
with open(log_path, "w") as f:
    json.dump(trades, f, indent=2)

print(f"💾 Trade log updated.")


💾 Trade log updated.
